In [7]:
# ------------------------------------------------------------
# Cell 1 – Setup folders & install PyYAML (if missing)
# ------------------------------------------------------------
# Install PyYAML only once; comment out if already in env
!pip -q install pyyaml

import pathlib, yaml, textwrap, pandas as pd, numpy as np
CONFIG_DIR = pathlib.Path("../provider_configs")
CONFIG_DIR.mkdir(exist_ok=True)

In [8]:
# ------------------------------------------------------------
# Cell 2 – Create price-sheet YAML (overwrite-safe)
# ------------------------------------------------------------
yaml_content = {
    "lease_fee" : {"Atom": 3.00, "Photon": 1.50, "Spin": 0.40},
    "exec_fee"  : {"Atom": 0.01, "Photon": 0.05, "Spin": 0.20},
    "trigger_fee": 0.01,
    "acq_cost"  : 0.20,
    "transfer_fee": {"Atom": 0.01, "Photon": 0.10, "Spin": 0.25}
}
config_path = CONFIG_DIR / "qpu_demo.yml"
with open(config_path, "w") as f:
    yaml.dump(yaml_content, f)
print("Config written →", config_path.resolve())

Config written → /Users/tej/Career/Projects/Quantum Compute Optimization/v1/provider_configs/qpu_demo.yml


In [9]:
code_str = """
import yaml, pathlib
from typing import Dict

class CostModel:
    \"\"\"Loads price sheet and exposes fee helpers.\"\"\"

    def __init__(self, yaml_path: str | pathlib.Path):
        with open(yaml_path, "r") as f:
            cfg = yaml.safe_load(f)
        self.lease_fee   : Dict[str, float] = cfg["lease_fee"]
        self.exec_fee    : Dict[str, float] = cfg["exec_fee"]
        self.trigger_fee : float            = cfg["trigger_fee"]
        self.acq_cost    : float            = cfg["acq_cost"]
        self.transfer_fee: Dict[str, float] = cfg["transfer_fee"]

    # ---------- granular fee look-ups ----------
    def lease(self, block_type: str, hours: int = 24, n_blocks: int = 1) -> float:
        return self.lease_fee[block_type] * hours * n_blocks

    def exec(self, block_type: str, n_jobs: int) -> float:
        return self.exec_fee[block_type] * n_jobs

    def trigger(self, n_jobs: int) -> float:
        return self.trigger_fee * n_jobs

    def acquisition(self, n_blocks: int) -> float:
        return self.acq_cost * n_blocks

    def transfer(self, new_type: str, n_blocks: int) -> float:
        return self.transfer_fee[new_type] * n_blocks
"""
path = pathlib.Path("cost_model.py")
path.write_text(textwrap.dedent(code_str))
print("Module saved →", path.resolve())

Module saved → /Users/tej/Career/Projects/Quantum Compute Optimization/v1/notebooks/cost_model.py


In [10]:
from cost_model import CostModel
cm = CostModel(config_path)

# Toy scenario: 2 Atom blocks leased today, each runs 1 workload
total = (
    cm.acquisition(2) +                 # 2 × 0.20
    cm.lease("Atom", hours=24, n_blocks=2) +  # 2 × 3.00 × 24
    cm.trigger(2) +                     # 2 × 0.01
    cm.exec("Atom", n_jobs=2)           # 2 × 0.01
)
print(f"Toy-day total cost: ${total:.2f}")
assert abs(total - (0.40 + 144 + 0.02 + 0.02)) < 1e-9
print("Unit-test ✔ passed")

Toy-day total cost: $144.44
Unit-test ✔ passed


In [13]:
import pyarrow.dataset as ds

blocks_ds = ds.dataset("../data/blocks.parquet")
wl_ds     = ds.dataset("../data/workloads_daily.parquet")

print("Total blocks rows :", blocks_ds.count_rows())

# ---- show 5 sample block sizes (version-safe) ----
sample = blocks_ds.scanner(columns=["qpu_units"]).head(5).to_pandas()
display(sample)

# ---- aggregate workloads for first 2 days ----
first2 = (wl_ds
          .filter(ds.field("day") < 2)
          .to_table(columns=["day","n_workloads"])
          .to_pandas()
          .groupby("day")["n_workloads"].sum())
display(first2)

Total blocks rows : 981230


,qpu_units
0,37417910
1,99702449
2,10674071
3,92483044
4,62600725


day
0    26268684
1    27178704
Name: n_workloads, dtype: int64